In [22]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from tqdm import tqdm

import numpy as np
import pandas as pd
import re, os, sys

import seaborn as sns
sns.set_style('white')

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
mpl.rcParams['pdf.fonttype'] = 42

import math

font_name = {'fontname':'Arial'}

plt.rcParams["font.family"] = "Arial"

import joblib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import cptac

In [3]:
pdac = cptac.Pdac()

cptac warning: Your version of cptac (1.1.2) is out-of-date. Latest is 1.5.7. Please run 'pip install --upgrade cptac' to update it. (/Users/yingweihu/opt/anaconda3/envs/py39_glycositeatlas/lib/python3.9/threading.py, line 910)


In [4]:
pdac.list_data()

Below are the dataframes contained in this dataset and their dimensions:

clinical
	238 rows
	39 columns
CNV
	140 rows
	19906 columns
derived_molecular
	140 rows
	49 columns
gene_fusion
	1212 rows
	7 columns
miRNA
	158 rows
	2416 columns
phosphoproteomics
	215 rows
	51469 columns
proteomics
	215 rows
	11662 columns
somatic_mutation
	6395 rows
	3 columns
transcriptomics
	161 rows
	28057 columns


In [5]:
#Let's get the proteomics data associated with Endometrial cancer; it was generated by the team at UMich
phospho = pdac.get_phosphoproteomics().T

In [6]:
phospho.head(2)

,,,Patient_ID,C3L-00017,C3L-00102,C3L-00189,C3L-00277,C3L-00401,C3L-00589,C3L-00598,C3L-00599,C3L-00622,C3L-00625,...,C3N-03069.N,C3N-03211.N,C3N-03426.N,C3N-03440.N,C3N-03780.N,C3N-03839.N,C3N-03840.N,C3N-03884.N,C3N-04119.N,C3N-04282.N
Name,Site,Peptide,Database_ID,,,,,,,,,,,,,,,,,,,,,
A1CF,T491,ITIPALASQNPAIHPFtPPK,NP_001185747.1,NaN,NaN,NaN,NaN,NaN,14.412375,NaN,14.722118,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.530723,NaN,NaN
A2M,S928,ETTFNSLLCPSGGEVsEELSLK,NP_000005.2,16.592797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,16.176262,NaN,NaN,NaN,NaN,NaN


In [7]:
GPNotebook_dir = r"/Users/yingweihu/Documents/GitHub/GPNotebook"
data_name = "PDAC"

wd = os.path.join(GPNotebook_dir,"sample",data_name)

# meta folder
meta_dir = os.path.join(wd,"meta") 
sample_path = os.path.join(meta_dir, "Supplementary_table_1_v3.0.xlsx")

In [8]:
wd

'/Users/yingweihu/Documents/GitHub/GPNotebook/sample/PDAC'

In [9]:
task_dir = os.path.join(wd,'result','GPNotebook','differential_expression_analysis')
if not os.path.exists(task_dir):
    os.mkdir(task_dir)

In [10]:
# input meta path: standardrized meta table
meta_path = os.path.join(wd,"meta/info.tsv")

# load meta data
meta_df = pd.read_csv(meta_path,sep="\t")
meta_df.head(10)

,Sample,CaseID,VitalStatus,SurvivalDays,Age,Gender,DeathCause,Stage
0,C3L-00102.N,C3L-00102,Deceased,249.0,42,Male,pancreatic carcinoma,Stage III
1,C3L-00189.N,C3L-00189,Deceased,1035.0,68,Female,pancreatic carcinoma,Stage IIB
2,C3L-00277.N,C3L-00277,Deceased,610.0,69,Male,pancreatic carcinoma,Stage IIB
3,C3L-00401.N,C3L-00401,Living,1228.0,62,Female,Unknown,Stage IIB
4,C3L-00640.N,C3L-00640,Living,594.0,59,Female,Unknown,Stage IIB
5,C3L-00819.N,C3L-00819,Deceased,602.0,74,Male,pancreatic carcinoma,Stage IIB
6,C3L-00881.N,C3L-00881,Living,3.0,80,Male,Unknown,Stage IIB
7,C3L-00928.N,C3L-00928,Deceased,761.0,58,Female,pancreatic carcinoma,Stage IIB
8,C3L-01031.N,C3L-01031,Living,369.0,64,Female,Unknown,Stage IB
9,C3L-01036.N,C3L-01036,Living,765.0,64,Male,Unknown,Stage IIB


In [11]:
tumor_samples = []
nat_samples = []

# read sample names
sample_df = pd.read_excel(sample_path,sheet_name="PDAC")

nd_samples = []

for index,row in sample_df.iterrows():
    status = row['Pathological.Status']
    sample = row['Sample.ID']
#     print(sample)
    if re.sub('PDA.','',sample) not in set(meta_df['Sample']) and status != 'Normal-duct':
        continue
    if status == "Tumor":
        tumor_samples.append(sample)
    elif status == 'NAT':
        nat_samples.append(sample)
    elif status == "Normal-duct":
        nd_samples.append(sample)
    else:
        print(sample)

In [12]:
phospho = phospho.map(lambda i: 0 if np.isinf(i) else float(i))

In [13]:
phospho.head(2)

,,,Patient_ID,C3L-00017,C3L-00102,C3L-00189,C3L-00277,C3L-00401,C3L-00589,C3L-00598,C3L-00599,C3L-00622,C3L-00625,...,C3N-03069.N,C3N-03211.N,C3N-03426.N,C3N-03440.N,C3N-03780.N,C3N-03839.N,C3N-03840.N,C3N-03884.N,C3N-04119.N,C3N-04282.N
Name,Site,Peptide,Database_ID,,,,,,,,,,,,,,,,,,,,,
A1CF,T491,ITIPALASQNPAIHPFtPPK,NP_001185747.1,NaN,NaN,NaN,NaN,NaN,14.412375,NaN,14.722118,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.530723,NaN,NaN
A2M,S928,ETTFNSLLCPSGGEVsEELSLK,NP_000005.2,16.592797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,16.176262,NaN,NaN,NaN,NaN,NaN


In [14]:
cols = [i if re.search('\.N',i) else i + ".T" for i in list(phospho.columns.values)]
phospho.columns = cols

In [15]:
tumor_samples = [re.sub('^PDA\.','',i) for i in tumor_samples]

In [16]:
nat_samples = [re.sub('^PDA\.','',i) for i in nat_samples]

In [18]:
from omicsone.plugins.diff import compare_two_groups

In [19]:
# calculate statistic test
diff = compare_two_groups(phospho,tumor_samples,nat_samples,method="Wilcoxon(Unpaired)",
                           max_miss_ratio_global=0.5, max_miss_ratio_group=0.5,fdr_cutoff=0.01, log2fc_cutoff=1)

51469it [00:14, 3574.07it/s]


In [20]:
diff_tsv = os.path.join(task_dir,'Phosphopeptide_Tumor_NAT_diff.tsv')
diff_joblib = os.path.join(task_dir, 'Phosphopeptide_Tumor_NAT_dfif.joblib')

In [23]:
diff.head(2)

,Log2FC(median),Log2FC(mean),Wilcoxon(Unpaired)(Stats),Wilcoxon(Unpaired)(P-value),FDR,-Log10(FDR),Significance
Feature,,,,,,,
"(AAAS, S462, IAHIPLYFVNAQFPRFsPVLGR;FsPVLGR;, NP_001166937.1)",2.498936,2.320312,6536.0,1.129116e-10,6.199782e-10,9.207624,S-U
"(AAGAB, S201, AFWMAIGGDRDEIEGLssDEEH, NP_001258814.1)",-0.089987,-0.123646,1280.0,2.778579e-01,3.181226e-01,0.497405,None


In [25]:
joblib.dump(diff, diff_joblib)

diff.to_csv(diff_tsv,sep="\t")